## 5. Point Registration

Point-based registration allows us to help the registration via pre-defined sets of corresponding points. The 'CorrespondingPointsEuclideanDistanceMetric' minimises the distance of between a points on the fixed image and corresponding points on the moving image. The metric can be used to help in a difficult registration task by taking into account positions are known to correspond. Think of it as a way of embedding expert knowledge in the registration procedure. We can manually or automatically select points via centroids of segmentations for example. Anything is possible.
Point sets should always be given to elastix with their corresponding image.

### Registration

In [20]:
# First import is currently necessary to run ITKElastix on MacOs
from itk import itkElastixRegistrationMethodPython
import itk
import numpy as np
from itkwidgets import compare, checkerboard, view

In order for 3D registration to work with a point set, the 'CorrespondingPointsEuclideanDistanceMetric', should be set as metric. For the 3D case, this means that the metric should be a multimetric with the first metric of type AdvancedImageToImageMetric and the second the 'CorrespondingPointsEuclideanDistanceMetric'. The Registration parameter should therefore be set to 'MultiMetricMultiResolutionRegistration', to allow a multimetric parameter.

In [21]:
# Import Images
fixed_image = itk.imread('data/CT_3D_lung_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_3D_lung_moving.mha', itk.F)

fixed_point_set = np.loadtxt('data/CT_3D_lung_fixed_point_set_corrected.txt', skiprows=2, delimiter=' ')
moving_point_set = np.loadtxt('data/CT_3D_lung_moving_point_set_corrected.txt', skiprows=2, delimiter=' ')

# Import and adjust Parameter Map
parameter_object = itk.ParameterObject.New()
parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid')
parameter_map_rigid['Registration'] = [
    'MultiMetricMultiResolutionRegistration']
original_metric = parameter_map_rigid['Metric']
parameter_map_rigid['Metric'] = [original_metric[0],
                                 'CorrespondingPointsEuclideanDistanceMetric']
parameter_object.AddParameterMap(parameter_map_rigid)

In [31]:
fixed_point_set

array([[-3.6351e+01, -3.6974e+01, -1.2045e+03],
       [-6.5037e+01,  2.1764e+01, -1.3445e+03],
       [-9.3723e+01, -4.5170e+01, -1.2795e+03],
       [-3.9083e+01, -9.4346e+01, -1.2395e+03],
       [-7.3233e+01, -1.4580e+00, -1.1945e+03],
       [-4.3181e+01, -1.5118e+01, -1.3045e+03],
       [-3.4985e+01, -2.8240e+00, -1.1895e+03],
       [-5.9573e+01, -6.0196e+01, -1.2995e+03],
       [-6.2305e+01, -2.7412e+01, -1.1895e+03],
       [-7.0501e+01, -1.7850e+01, -1.3145e+03],
       [-7.1867e+01, -7.5222e+01, -1.2195e+03],
       [-5.2743e+01,  2.0398e+01, -1.2545e+03],
       [-5.9573e+01, -9.2000e-02, -1.2995e+03],
       [-4.5913e+01, -1.9216e+01, -1.2245e+03],
       [-5.4109e+01, -3.8340e+01, -1.2795e+03],
       [-7.3233e+01, -4.6536e+01, -1.2745e+03],
       [-5.1377e+01, -5.3366e+01, -1.2095e+03],
       [-8.8259e+01, -2.3314e+01, -1.2345e+03],
       [-4.5913e+01,  1.0836e+01, -1.3445e+03],
       [-1.9959e+01, -1.6484e+01, -1.1645e+03],
       [-2.8155e+01, -2.7412e+01, -1.329

In [28]:

def createPointSetITK(point_set):
    PixelType = itk.F
    Dimension = 3

    PointSetType = itk.Mesh[PixelType, Dimension]
    PointSet = PointSetType.New()

    points = PointSet.GetPoints()
    i = 0
    # Create points
    for a,b,c in point_set:        
        p = itk.Point[PixelType, Dimension]()
        p[0] = a
        p[1] = b
        p[2] = c
        points.InsertElement(i, p)
        i += 1
    
    return PointSet

itk_fixed_point_set = createPointSetITK(fixed_point_set)


In [27]:
itk_fixed_mesh = itk.meshread('data/CT_3D_lung_fixed_point_set_corrected.txt', itk.F)

RuntimeError: No MeshIO is registered to handle the given file.

In [32]:
itk.meshwrite(itk_fixed_point_set, 'outputpointset.vtk')

In [18]:
print(itk_fixed_point_set)

PointSet (000001D9D1E98450)
  RTTI typeinfo:   class itk::PointSet<float,3,class itk::DefaultStaticMeshTraits<float,3,3,float,float,float> >
  Reference Count: 1
  Modified Time: 237
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  Number Of Points: 51
  Requested Number Of Regions: 0
  Requested Region: -1
  Buffered Region: -1
  Maximum Number Of Regions: 1
  Point Data Container pointer: 0000000000000000
  Size of Point Data Container: 0



The point sets can be visualized and analysed with the itkwidgets package. Currently the point set visualization layer is put on top of the 3D image, which makes the slice view show all 'passed' points as well, this will change in future versions of the itkwidgets. (Changing the point set size and the gradient opacity of the image aids the visualization of the point sets in the 3D image)

In [4]:
view(fixed_image, point_sets=[fixed_point_set])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157, 0.       , 0.       ]], dtype…

In [5]:
view(moving_image, point_sets=[moving_point_set])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157, 0.       , 0.       ]], dtype…

With regards to the elastix function, the point sets do not need to be initialized first, so their file name + path can be given directly to elastix. Future version of ITKElastix will support initialization of point sets and passing these to elastix, just like the point set was initialized for the visualization with the itkwidgets.

Registration can either be done in one line with the registration function...

In [128]:
# Call registration function
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    fixed_point_set_file_name='data/CT_3D_lung_fixed_point_set_corrected.txt',
    moving_point_set_file_name='data/CT_3D_lung_moving_point_set_corrected.txt',
    log_to_console=False,
    parameter_object=parameter_object)

.. or by initiating an elastix image filter object.

In [111]:
# Load Elastix Image Filter Object
# Fixed and moving image should be given to the Elastix method to ensure that
# the correct 3D class is initialized.
elastix_object = itk.ElastixRegistrationMethod.New(fixed_image,moving_image)
elastix_object.SetFixedPointSetFileName('data/CT_3D_lung_fixed_point_set_corrected.txt')
elastix_object.SetMovingPointSetFileName(
    'data/CT_3D_lung_moving_point_set_corrected.txt')
elastix_object.SetParameterObject(parameter_object)
elastix_object.SetLogToConsole(False)

# Update filter object (required)
elastix_object.UpdateLargestPossibleRegion()

# Results of Registration
result_image = elastix_object.GetOutput()
result_transform_parameters = elastix_object.GetTransformParameterObject()